# Simulations

This notebook contains the code to generate ergodicity experiment simulations. Basic idea behind these simulations is that we want to **find optimal experimental design maximizing the disagreement between competing theories**. Here, we are able to simulate different isoelastic agents facing different wealth dynamics. According to ergodicity economics, time-optimal agent should use an isoelastic utility function with risk parameter equal to risk parameter generating wealth dynamics, to maximize growth of its wealth over time. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
import json

from os.path import join
from pathlib import Path
from tqdm.notebook import tqdm
from itertools import combinations, combinations_with_replacement, product
from utils.plotting import aligned_imshow_cbar
from utils.paralell import (agent_name, calculate_beta, isoelastic_utility, 
                            inverse_isoelastic_utility, wealth_change, 
                            shuffle_along_axis, create_gambles, 
                            create_gamble_pairs, create_trial_order, 
                            create_experiment, is_mixed, is_nobrainer, 
                            is_equal_growth, disagreement, bankruptcy_chance, 
                            richness_chance, experiment_duration, 
                            run_simulation, softmax, create_dmag_thrs, 
                            create_mag_thrs, create_var_thrs, 
                            is_g_deterministic)                         

## Modeling the agent

Assumptions about the agent:
1. Agent perfectly knows how different wealth changes influence his current wealth.
    - In real experiment this assumption is realized during the passive session in which subjects learn how different wealth changes influence dynamics. Obviously, real participant cannot perfectly know the effect of wealth change, but they can approximate it well enough to make appropriate choices during the active phase.
2. Agent tracks his current wealth after each trial.
    - This assumption may not be met if the improved experiment would not involve visible wealths (as original Copenhagen experiment). It is still a matter of debate how hidden wealth should be treated. There are two possible solutions. First, we can assume that agents calculate their choices as if they are constantly endowed with inital wealth $x_0$. On the other hand, choosing the gambles give (at least theoretically) access to possible wealth trajectories. However, after just a few trials the space of possible trajectories becomes to vast to computationally track. Experiment with hidden wealth assuming constant wealth level is very easy to optimize, because it lacs *wealth-trajectory-dependent* effects on choice. Inspection of $\eta^*$ plots lead to the observation that choice preference depends on agent's wealth for all dynamics other than multiplicative.    
3. At each trial agent is facing two gambles and it has to choose one of them.
4. Agent is computing expected change in utility for both gambles to guide his choice.
    - This is essential part of the behavioral model. Let's assume that agent is facing a pair of gambles: $g_a=(\gamma^a_1, \gamma^a_2)$, and $g_b=(\gamma^b_1, \gamma^b_2)$. Lets also assume that agent's risk attitude is $\eta_{\text{agent}}$ and the wealth dynamics is given by $\eta_{\text{dynamic}}$. Lets denote agent's utility function (the Isoelastic utility with $\eta=\eta_{\text{agent}}$) as $u$, and time-optimal utility function (the Isoelastic utility with $\eta=\eta_{\text{dynamict}}$) as $u^*$. An agent first computes utility of his current wealth $u(x_t)$. Then, for each gamble he calculates expected utility for this gamble as:
    $$E[u^{a/b}(x_{t+1})]=\frac{1}{2}(u(u_*^{-1}(u_*(x_t)+\gamma^{a/b}_1))+u(u_*^{-1}(u_*(x_t)+\gamma^{a/b}_2))).$$
Then, the agent computes expected change in utility for $g_a$ and $g_b$ as:
    $$E[\Delta u^{a/b}]=E[u^{a/b}(x_{t+1})-u(x_t)]=E[u^{a/b}(x_{t+1})]-u(x_t)$$
> Note that for the time-optimal agent $u=u_*$, so $E[u^{a/b}(x_{t+1})]=u(x_t)+\frac{\gamma^{a/b}_1+\gamma^{a/b}_2}{2}$. This implies that $E[\Delta u^{a/b}]=\frac{\gamma^{a/b}_1+\gamma^{a/b}_2}{2}$, so expected change in utility for a gamble reflects true average growth rate for that gamble. 

5. These expected changes in utility are passed through [Softmax choice function](https://en.wikipedia.org/wiki/Softmax_function) to simulate choice stochasticity. 
    - Probability of choosing gamble $g_a$ is given as:
    $$P(a)=\exp^{-1}(\beta(E[\Delta u^a]-E[\Delta u^b]))$$
    - Since different agents use different utility function, softmax precision parameter $\beta$ is normalized between agents. Normalized precision is a precision for which an agent endowed with initial wealth $x0$ and facing two extreme, opposite fractals with growth rates $-c$, and $c$ would choose the winning fractal with probability $p_{\text{threshold}}$. Function `calculate_beta` can be used to find normalized precision for agent with risk attitude $\eta$. Normalized precision depends on agent's risk attitude, wealth dynamic, intial wealth, growth rate scaling $c$, and probability threshold specified by the user.
6. Softmax function return choice probabilities for both gambles that are used to generate actual choice.

## Modeling an experiment

Assumptions about the experiment:
1. Each experiment consists of specified number of trials, $N_{\text{trials}}$, contolled by the `n_trials` variable.
2. Any number of gamble pairs can be used to generate the experiment. 
    - If the number of available gamble pairs is less than $N_{\text{trials}}$, gamble pairs are repeated to fill the duration of the experiment. For example, constraining gamble pair space to 100 unique gamble pairs with $N_{\text{trials}}=360$ would lead repeating each gamble pair 3 or 4 times (60 gamble pairs repeated one extra time are randomly selected). The order of gambles is random.
    - If the number of available gamble pairs is greater than $N_{\text{trials}}$, a random subset of $N_{\text{trials}}$ gamble pairs is selected for each simulated run. Different runs would result in different subset, so each available gamble pair has equal change of being selected.   
3. Available gamble pairs constituing an experiment are selected by geometrically constraining gamble and gamble pair space (see notebook `ee_03`).      
4. Single simulated run can be prematurely terminated if agent's wealth drop below 0 or raise above $x_{\text{limit}}$.
    - This assumption introduces lower and upper bounds to agents wealth. This is very pragmatic as it provides simple means of controlling cost, but has some associated caveats. In particular it has been shown that time-optimal behavior changes close to the wealth bounds. Modeling this termination mechanism also benefits disagreement measure. Terminated trajectories, i.e., experiments that ended prematurely have less contribution to overall disagreement. In this way, disagreement not only quantifies our ability to discriminate agents, but also is sensible to cost problems for experiments with excessive wins or losses.
5. For each setup comprised of fixed: gamble pair set, growth rate scaling $c$, agent risk attitude $\eta_{\text{agent}}$ multiple runs are simulated in parallel. We will refer to this set of simulations as to *parallel run*.
    - Simulating multiple realizations is important because of the wealth-trajectory-dependency problem. By realizing same setup multiple times we can get more accurate approximation of true expected disagreement. Number of performed run is controlled by the `n_simulations` variable.
  
## Gamble space constraints

Additional assumptions about constraining gamble space (specific to the second version of the simulation):
1. $\gamma_{\text{min}}$ and $\gamma_{\text{max}}$ are included but only first seven bound positions are considered. The seventh position corresponds to the setup for which all but one win-only or loss-only gambles are removed. This limitation is imposed to prevent excessive imbalance between win-only and loss-only gambles.
2. $var_{\text{min}}$ and $var_{\text{min}}$ are excluded but the setting for $var_{\text{min}}$ is fixed to 1, i.e. first possible bound placement excluding all deterministic gambles. Deterministic gambles are excluded to prevent imbalances resulting from subject's aversion/liking towards gambling. 
3. $\Delta \gamma_{\text{max}}$ is included with all possible levels, but $\Delta \gamma_{\text{min}}$ is excluded. Since $\eta^*$ plots showed that discrepant trials are often comprised of two gambles with similar average growth rates, it is more important to remove gamble pairs with high $\Delta \gamma$. 
4. $\Delta var_{\text{min}}$ and $\Delta var_{\text{max}}$ are excluded.
> In future simulations it might be worth to include $\Delta var_{\text{min}}$, since $\eta^*$ plots showed that discrepant trials are often comprised of two gambles with diffent variance.
5. No-brainer gamble pairs are excluded.
6. Range of growth rate scaling depends on dynamic.
    - For risk-seeking dynamic $c$ ranges from 1 to 300000 (step 3000).
    - For additive dynamic $c$ ranges from 1 to 300 (step 3).
    - For risk-seeking dynamic $c$ ranges from 0.01 to 0.3 (step 0.003).
  
## Collected statistics

After each parallel run, several statistics are collected and stored for further analysis. Individual choices and wealth trajectories are then discarded to free space for another simulation. Collected statistics:
- `d_p`: *Softmax disagreement*. It is calculated for every pair of simulated agents. It reflects average difference in choice frequency for two Isoelastic agents.
- `d_y`: *Choice disagreement*. It reflects the proportion of trials for which two agents selected different gambles. However, this is less useful than `d_p` because `d_y` generally have two sources: one reflecting pure randomness and the other reflecting true difference in choice preference. To better understand that imagine both agents having equal choice probability for both options. In this case they would still select different option in 50% times just because of pure randomness. Notice that this situation is useless from the perspective of discriminating competing models. 
- `p_bank`: *Bankruptcy probability*. Proportion of wealth trajectories that ended before the last planned trial because of agent's bankruptcy. It is calculated for every simulated agent independently.
- `p_rich`: *Richness probability*. Proportion of wealth trajectories that ended before the last planned trial because of agent exceeded the upper wealth limit.
- `avg_len`: *Expected duration of the experiment*. Average lenght of trajectory. If its equal to $N_{\text{trials}}$, then all ended as planned. 
- `n_gamble_pairs`: *Number of unique gamble pairs*. Number of available gamble pairs (trials) for a particular experimental setup. It only depends on geometrical constraints acting on gamble and gamble pair space.

## Simulation settings
- `x_0`: Initial endowment.
- `x_limit`: Upper wealth limit used to terminate runs that exceeded that limit. 
- `n_fractals`: Number of distinct wealth changes (fractals).
- `n_trials`: Total number of experimental trials.
- `p_threshold`: Critial choice probaiblity used to normalize softmax sensitivity across agents. Normalization ensures that all isoelastic agents would choose the best fractal over the worst fractal with `p_threshold` probability. Values closer to one indicate more sensitive agents and more deterministic choices.
- `agents`: List of simulated agents. Values correspond to agent's risk attitude.
- `eta_dynamic`: Risk attitude constituing wealth dynamic. 
- `c_min`: Minimum growth rate scaling. It should be adjusted for each dynamic separately.
- `c_max`: Maximum growth rate scaling. It should be adjusted for each dynamic separately.
- `n_c`: Number of samples within growth scaling range $[c_{\text{min}}, c_{\text{max}}]$.
- `min_unique_gambles`: Required number of unique gamble pairs to construct the experiment. All setups with number of unique gamble pairs less than specified would not be evaluated.
- `max_mag_n`: Maximum setting for upper and lower average growth rate bounds. Ensure that setup is not too imbalance toward winning or loosing gambles. 
- `n_simulations`: Number of individual runs performed within single parallel run.
- `wealth_dependency`: Decision if wealth dependency effects should be taken into account. Setting this flag to `False` is appropriate for simulating experiment with hidden wealth.

In [ ]:
x_0 = 1000
x_limit = 4000
n_fractals = 9
n_trials = 360
p_threshold = 0.9999
agents = [-1, 0, 1]

# Gamble space settings 
eta_dynamic = 1
c_min = 0.001
c_max = 0.3
n_c = 100
min_unique_gambles = int(360 / 4)
max_mag_n = 7 

# Other
n_simulations = 1000
wealth_dependency = True

In [ ]:
n_mag_thrs = max_mag_n + 1
n_dmag_thrs = 2 * n_fractals - 2

# Store number of gambles after filtering
n_pairs = np.zeros((n_mag_thrs, n_mag_thrs, n_dmag_thrs), dtype=int)

mag_thrs = create_mag_thrs(1, n_fractals)
dmag_thrs = create_dmag_thrs(1, n_fractals)
for ml_idx, mh_idx in product(range(n_mag_thrs), repeat=2):
    for md_idx in range(n_dmag_thrs):
        
        ml_bound = mag_thrs[ml_idx]
        mh_bound = mag_thrs[-1 - mh_idx]
        md_bound = dmag_thrs[md_idx]

        gambles = create_gambles(1, n_fractals)
        gambles = [
            g for g in gambles 
            if np.mean(g) > ml_bound
            and np.mean(g) < mh_bound
            and not is_g_deterministic(g)
        ]
        
        gamble_pairs = create_gamble_pairs(gambles)
        gamble_pairs = [
            gp for gp in gamble_pairs
            if np.abs(np.mean(gp[0]) - np.mean(gp[1])) < md_bound
            if not is_nobrainer(gp)
        ]
        
        n_pairs[ml_idx, mh_idx, md_idx] = len(gamble_pairs)
    
    # remove redundant search values
    cutoff = np.where(np.diff(n_pairs[ml_idx, mh_idx]) == 0)[0]
    if len(cutoff):
        n_pairs[ml_idx, mh_idx, cutoff[0] + 1:] = 0
        cutoff_calc = 2 * n_fractals - 2 - ml_idx - mh_idx + int(ml_idx + mh_idx >= n_fractals - 1)
        
# remove max and min corners due to lower variance threshold
n_pairs[0, :] = 0
n_pairs[:, 0] = 0

# remove setups with not enough unique gamble pairs
n_pairs[n_pairs < min_unique_gambles] = 0

print("Gamble pair space subset: ", np.sum(n_pairs != 0))

In [ ]:
path_output = join("data", "v2", f"simulation_{agent_name(eta_dynamic)}")
Path(path_output).mkdir(exist_ok=False)

In [ ]:
n_agents = len(agents)

shape_common = (n_mag_thrs, n_mag_thrs, n_dmag_thrs, n_c)
shape_pair = (math.comb(n_agents, 2), ) + shape_common
shape_single = (n_agents, ) + shape_common

# Statistics
d_p = np.zeros(shape_pair)
d_y = np.zeros(shape_pair)
p_bank = np.zeros(shape_single)
p_rich = np.zeros(shape_single)
avg_len = np.zeros(shape_single)
n_gamble_pairs = np.zeros(shape_common)

c_range = np.linspace(c_min, c_max, n_c)
for c_idx, c in tqdm(list(enumerate(c_range))):
    
    mag_thrs = create_mag_thrs(c, n_fractals)
    dmag_thrs = create_dmag_thrs(c, n_fractals)
    
    for ml_idx, mh_idx in product(range(n_mag_thrs), repeat=2):
        for md_idx in range(n_dmag_thrs):
                        
            if n_pairs[ml_idx, mh_idx, md_idx] == 0:
                # Ignore excluded parameter values
                continue
        
            # Translate bound indices into average growth rates
            ml_bound = mag_thrs[ml_idx]
            mh_bound = mag_thrs[-1 - mh_idx]
            md_bound = dmag_thrs[md_idx]

            # Create & filter gambles 
            gambles = create_gambles(c, n_fractals=n_fractals)
            gambles = [
                g for g in gambles 
                if np.mean(g) > ml_bound
                and np.mean(g) < mh_bound
                and not is_g_deterministic(g)
            ]

            # Create & filter gamble pairs
            gamble_pairs = create_gamble_pairs(gambles)
            gamble_pairs = [
                gp for gp in gamble_pairs 
                if np.abs(np.mean(gp[0]) - np.mean(gp[1])) < md_bound
                if not is_nobrainer(gp)
            ]

            # Create experiment and randomized trial order
            experiment = create_experiment(gamble_pairs)
            trial_order = create_trial_order(
                n_simulations, 
                experiment.shape[-1], 
                n_trials
            )

            x, y, p = {}, {}, {}
            for agent in agents:

                # Estimate precision
                beta = calculate_beta(
                    eta_dynamic=eta_dynamic,
                    eta_agent=agent,
                    x_0=x_0,
                    x_limit=x_limit,
                    p_threshold=p_threshold,
                    c=c,
                )

                x[agent], p[agent], y[agent] = run_simulation(
                    experiment=experiment,
                    trial_order=trial_order,
                    eta_dynamic=eta_dynamic,
                    eta_agent=agent,
                    x_0=x_0,
                    x_limit=x_limit,
                    beta=beta,
                    wealth_dependency=wealth_dependency
                )

            # Store statistics
            for i, (a1, a2) in enumerate(combinations(agents, 2)):
                d_p[i, ml_idx, mh_idx, md_idx, c_idx] = disagreement(p[a1], p[a2])        
                d_y[i, ml_idx, mh_idx, md_idx, c_idx] = disagreement(y[a1], y[a2])
            for i, a in enumerate(agents):
                p_bank[i, ml_idx, mh_idx, md_idx, c_idx] = bankruptcy_chance(x[a])
                p_rich[i, ml_idx, mh_idx, md_idx, c_idx] = richness_chance(x[a], x_limit)
                avg_len[i, ml_idx, mh_idx, md_idx, c_idx] = experiment_duration(x[a], x_limit)

In [ ]:
# Store main data
np.save(join(path_output, "d_p.npy"), d_p)
np.save(join(path_output, "d_y.npy"), d_y)
np.save(join(path_output, "p_rich.npy"), p_rich)
np.save(join(path_output, "p_bank.npy"), p_bank)
np.save(join(path_output, "avg_len.npy"), avg_len)
np.save(join(path_output, "n_pairs.npy"), n_pairs)

# Create and store metadata
dimensions = {
    0: list(combinations(agents, 2)),
    1: list(create_mag_thrs(1, n_fractals)[:n_mag_thrs]),
    2: list(create_mag_thrs(1, n_fractals)[-n_mag_thrs:]),
    3: list(create_dmag_thrs(1, n_fractals)),
    4: list(c_range)
}

meta = {
    "x_0": x_0,
    "x_limit": x_limit,
    "n_fractals": n_fractals,
    "n_trials": n_trials,
    "p_threshold": p_threshold,
    "agents": agents,
    "eta_dynamic": eta_dynamic,
    "c_min": c_min,
    "c_max": c_max,
    "n_c": n_c,
    "min_unique_gambles": min_unique_gambles,
    "max_mag_n": max_mag_n,
    "n_simulations": n_simulations,
    "wealth_dependency": wealth_dependency,
    "dimensions": dimensions
}

with open(join(path_output, "metadata.json"), "w") as f:
    f.writelines(json.dumps(meta, indent=2))